In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **IMPORTING LIBRARIES FOR THE MODEL**

---

---



In [ ]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score

#For Saving the Model
import joblib

# **DATA DESCRIPTION**

---



---



In [ ]:
df = pd.read_csv("/content/drive/MyDrive/FRAUD DETECTION/SOURCE/dataset_final.csv")

FileNotFoundError: ignored

In [ ]:
df.head(2).T

NameError: ignored

In [ ]:
df.shape

(1000000, 49)

   DROPPING AN UNNECESSARY COLUMN

In [ ]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
df.shape

(1000000, 48)

In [ ]:
#X=df.drop(["fraud_bool"],axis=1)
#Y=df["fraud_bool"]
#x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.30,random_state=42,stratify=y)

 **PERFORMING RANDOM UNDER_SAMPLING FOR DATA**


---



---





In [ ]:
count_class_0, count_class_1 = df.fraud_bool.value_counts()

# Divide by class
df_class_0 = df[df['fraud_bool'] == 0]
df_class_1 = df[df['fraud_bool'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample((count_class_1)*2)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.fraud_bool.value_counts())

Random under-sampling:
0.0    22058
1.0    11029
Name: fraud_bool, dtype: int64


**PERFORMING TRAIN-TEST SPLIT OF DATASET FOR MODEL EVALUATION**


---



---





In [ ]:
X = df_test_under.drop('fraud_bool',axis='columns')
y = df_test_under['fraud_bool']

x_train_sample, x_test_sample, y_train_sample, y_test_sample = train_test_split(X, y, test_size=0.2,random_state=42)

In [ ]:
print(x_train_sample.shape)
print(x_test_sample.shape)

(26469, 47)
(6618, 47)


In [ ]:
weights_assigned={0:1,1:2}

In [ ]:
n_inputs=(len(x_train_sample.columns))

In [ ]:
model_ANN = keras.Sequential()
model_ANN.add(keras.layers.BatchNormalization(input_shape=[x_train_sample.shape[1]]))
model_ANN.add(keras.layers.Dense(128,input_dim=n_inputs,activation='relu',kernel_initializer='he_uniform'))
#kernal_initializer='he_uniform' ---> weight initializer for relu 
model_ANN.add(keras.layers.Dropout(0.5))
model_ANN.add(keras.layers.BatchNormalization())
model_ANN.add(keras.layers.Dense(64,activation='relu'))
model_ANN.add(keras.layers.Dropout(0.5))
model_ANN.add(keras.layers.BatchNormalization())
model_ANN.add(keras.layers.Dense(64,activation='relu'))
model_ANN.add(keras.layers.Dropout(0.5))
model_ANN.add(keras.layers.BatchNormalization())
model_ANN.add(keras.layers.Dense(32,activation='relu'))
model_ANN.add(keras.layers.Dropout(0.5))
model_ANN.add(keras.layers.Dense(1,activation='sigmoid'))
model_ANN.compile(loss='binary_crossentropy',optimizer='ADAM')
#class_weight=weights_assigned
model_ANN.fit(x_train_sample,y_train_sample,epochs=100,batch_size=1032)

Epoch 1/10
26/26 [==============================] - 4s 30ms/step - loss: 1.0705
Epoch 2/10
26/26 [==============================] - 1s 25ms/step - loss: 0.9428
Epoch 3/10
26/26 [==============================] - 1s 19ms/step - loss: 0.8708
Epoch 4/10
26/26 [==============================] - 1s 20ms/step - loss: 0.8078
Epoch 5/10
26/26 [==============================] - 1s 36ms/step - loss: 0.7637
Epoch 6/10
26/26 [==============================] - 2s 59ms/step - loss: 0.7384
Epoch 7/10
26/26 [==============================] - 1s 41ms/step - loss: 0.7122
Epoch 8/10
26/26 [==============================] - 1s 41ms/step - loss: 0.7015
Epoch 9/10
26/26 [==============================] - 1s 44ms/step - loss: 0.6894
Epoch 10/10
26/26 [==============================] - 1s 45ms/step - loss: 0.6766


In [ ]:
y_pred=model_ANN.predict(x_test_sample)

207/207 [==============================] - 1s 2ms/step


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np

In [ ]:
 print("ROC_AUC Details:",roc_auc_score(y_test_sample,y_pred))

ROC_AUC Details: 0.8712025098599604


In [ ]:
#Predicting  on Models
y_pred01=np.round(y_pred)
y_pred01=y_pred01.astype(int)

In [ ]:
#Printung Classification Report
print(classification_report(y_test_sample, y_pred01))

              precision    recall  f1-score   support

         0.0       0.88      0.81      0.84      4372
         1.0       0.68      0.78      0.73      2246

    accuracy                           0.80      6618
   macro avg       0.78      0.79      0.78      6618
weighted avg       0.81      0.80      0.80      6618



In [ ]:
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
print(confusion_matrix(y_test_sample,y_pred01))

[[3555  817]
 [ 504 1742]]


As we can see, model_ANN is performing well against fraud , so we will use this model for deployment

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm= SVC(kernel='linear', random_state=5)  
svm.fit(x_train_sample,y_train_sample)
pred_04= svm.predict(x_test_sample)
print(classification_report(y_test_sample,pred_04))
print("roc auc:",roc_auc_score(y_test_sample,pred_04)) 

              precision    recall  f1-score   support

         0.0       0.80      0.81      0.81      2232
         1.0       0.80      0.79      0.80      2180

    accuracy                           0.80      4412
   macro avg       0.80      0.80      0.80      4412
weighted avg       0.80      0.80      0.80      4412

roc auc: 0.8015668672519812


# **Saving the Model**

In [ ]:
joblib.dump(model_ANN,'/content/drive/MyDrive/FRAUD DETECTION/MODELS/ANN.joblib')

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......batch_normalization
.........vars
............0
............1
............2
............3
......batch_normalization_1
.........vars
............0
............1
............2
............3
......batch_normalization_2
.........vars
............0
............1
............2
............3
......batch_normalization_3
.........vars
............0
............1
............2
............3
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dropout
.........vars
......dropout_1
.........vars
......dropout_2
.........vars
......dropout_3
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
....

['/content/drive/MyDrive/FRAUD DETECTION/MODELS/ANN.joblib']